In [1]:

import pandas as pd
import numpy as np

import json
import os

import re
import pymongo

from credentials_local import user, password, rds_host


from boston_functions import *
from fractions import Fraction
import re

from liquid import liquids
from garnish import garnishes
import random

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  


In [3]:
#import mr boston cocktail database
data = pd.read_csv("mr-boston-all-glasses.csv")
#locate data
data = data[data.loc[:, "glass-size"].notna()]

valid_units = ["oz", "tsp", "splash", "dash"]

fill_liquid = ["ginger ale", "carbonated water", "cola", "water", "chilled champagne", "soda water", 
               "club soda", "ginger ale or soda water", "lemon-lime soda", "ginger beer", "bitter lemon soda",
               "apple juice", "orange juice"]

invaild_ingredients = ['chopped', 'cut in half', 'cut in halves', 'cut into halves', 'flamed', 'hulled', 'long',
                       'skinned','whipped', "preferably b.a. reynold's", 'preferably jamaican', 'preferably japanese', 
                       'preferably pedro ximenez', "such as bittermen's elemakule", 'such as demerara', 
                       'such as islay or skye', 'such as nasturtium']

ingredient_indices = range(3, 14)

#get all recipe data
all_recipes, measures = get_cocktail_recipes(data, ingredient_indices, liquids, garnishes, invaild_ingredients, valid_units)

#set categories list
categories = list(set(data.iloc[:, 1])) + ["AI Instant Classic"]

In [4]:
############################
# COCKTAIL RECIPES TO TEXT #
############################
with open('mr_boston_cocktails.txt', 'w') as text_file:
    for i in range(len(all_recipies)):
        cocktail_string = all_recipies[i]['name'] + " - "
        cocktail_string += all_recipies[i]['glass'] + " - "
        for measurement, ingredient in all_recipies[i]["recipe"]:
            cocktail_string += measurement + " " + ingredient + " - "
        cocktail_string += all_recipies[i]['instructions']
        text_file.write(cocktail_string + "\n")

NameError: name 'all_recipies' is not defined

In [5]:
###################
# POPULATE TABLES #
###################

####################
### LIQUID TABLE ###
####################
#import colors for liquids
liquid_colors = pd.read_csv("Liquid_Colors_Final.csv")



In [6]:

### GLASS SVG DATA ###
#import glass svg data
svg_data = pd.read_csv("the_glasses.csv")
svg_dict_list = svg_data.to_dict(orient='records')


In [7]:
for svg in svg_dict_list:
    if svg['name'] == "Beer Mug":
           svg['name'] = "fluted_lager"
    elif (svg['name'] == "Brandy Snifter") or (svg['name'] == "Red-Wine Glass") or (svg['name'] == "White-Wine Glass"):
        svg['name'] = "poco_grande"                                
    elif svg['name'] == "Champagne Flute":
        svg['name'] = "champagne_extra_fluted"                
    elif svg['name'] == "Cocktail Glass":
        svg['name'] = "cocktail_lg"
    elif svg['name'] == "Collins Glass":
        svg['name'] = "collins"
    elif (svg['name'] == "Cordial or Pony Glass") or (svg['name'] == "Pousse-Cafe Glass") or (svg['name'] == "Sherry Glass"):
        svg['name'] = "pousse_cafe"
    elif svg['name'] == "Highball Glass":
        svg['name'] = "highball"
    elif svg['name'] == "Hurricane Glass":
        svg['name'] = "hurricane"
    elif svg['name'] == "Irish Coffee Glass":
        svg['name'] = "irish_coffee"
    elif svg['name'] == "Old-Fashioned Glass":
        svg['name'] = "old_fashioned"
    elif (svg['name'] == "Punch Cup") or (svg['name'] == "Shot Glass"):
        svg['name'] = "rocks"
    elif svg['name'] == "Sour Glass":
        svg['name'] = "champagne_flute"

In [8]:
all_recipes[0]

{'name': 'Gauguin',
 'category': 'Cocktail Classics',
 'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Top with a cherry.',
 'glass': 'Old-Fashioned Glass',
 'glass_size': 6,
 'recipe': [['2 oz', 'batavia arrack or light rum'],
  ['1 oz', 'passion fruit syrup'],
  ['1 oz', 'fresh lemon juice'],
  ['1 oz', 'lemon or lime juice']]}

In [9]:
glasses = []
glass_dicts = []
for recipe in all_recipes:
    if recipe['glass'] not in glasses:
        this_glass = {}
        this_glass['glass_name'] = recipe['glass']
        this_glass['glass_size'] = recipe['glass_size']
        glasses.append(recipe['glass'])
        glass_dicts.append(this_glass)
glass_df = pd.DataFrame(glass_dicts)
glass_df
        



,glass_name,glass_size
0,Old-Fashioned Glass,6
1,Cocktail Glass,6
2,Collins Glass,14
3,Champagne Flute,6
4,Cordial or Pony Glass,2
5,Highball Glass,7
6,Irish Coffee Glass,8
7,Beer Mug,10
8,Sour Glass,5
9,Brandy Snifter,8


In [12]:

for recipe in all_recipes:
    for svg in svg_dict_list:
        if recipe['glass'] == svg['name']:
            recipe['Mask'] = svg['mask']
            recipe['Path'] = svg['path']
            recipe['Mask_Height'] = svg['maskHeight']
            recipe['Mask_Top_Margin'] = svg['maskTopMargin']

In [13]:
svg_masks = list(svg_data['mask'])


In [14]:
all_recipes_copy = all_recipes

In [15]:
for recipe in all_recipes_copy:
    recipe['Ingredients'] = []
    
    #find total glass volume
    total_glass_volume = recipe["glass_size"]
    #set total liquid measure to 0
    total_liquid_measure = 0
    for measure, ingredient in recipe['recipe']:
        measure = measure.strip()
        if measure == "add":
            recipe['Garnish_Instructions'] = measure
            recipe['Garnish'] = ingredient
#         elif ingredient == "pimm's cup":#pimm's cup did not make it onto out liquids list
#             pass

        else:
            this_ing = {}
            #add to lists
            this_ing['Liquid'] = ingredient
            this_ing['Measure'] = measure
        ### CONVERTING MEASURES TO FLOAT ###
            #split measure on spaces
            split_measure = measure.split(" ")

            #if measure is compound fraction,
            if len(split_measure) > 2:
                #recombine compound fraction
                measure = split_measure[0]+" "+split_measure[1]
                #convert to float
                measure_float = float(sum(Fraction(s) for s in measure.split()))
                #add to dict
                this_ing['Measure_Float'] = measure_float
                #add measure to drink's total measure
                total_liquid_measure += measure_float
            #if measure is an integer,
            elif len(split_measure) > 1:
                #convert to float
                measure_float = float(sum(Fraction(s) for s in split_measure[0].split()))
                #if unit is smaller than oz, convert measure to oz
                if split_measure[1][:3] == "tsp":
                    measure_float = measure_float / 6
                elif split_measure[1][:4] == "dash":
                    measure_float = measure_float / 32
                elif split_measure[1][:6] == "splash":
                    measure_float = measure_float / 5
                #add to float list
                this_ing['Measure_Float'] = measure_float
                #add measure to drink's total measure
                total_liquid_measure += measure_float
            #if measure if fill,
            elif split_measure[0] == "fill":
                #calculate remaining volume
                remaining_volume = float(total_glass_volume) - float(total_liquid_measure)
                #set fill measure to half remaining volume
                measure_float = float(remaining_volume / 2)
                #add to float list
                this_ing['Measure_Float'] = measure_float
            else:
                measure_float = float(0)
                this_ing['Measure_Float'] = measure_float
        recipe['Ingredients'].append(this_ing)
       

In [16]:
for recipe in all_recipes_copy:
#     print(recipe['name'])
    vols = []
    for ingredient in recipe['Ingredients']:
#         print(ingredient)
        vols.append(ingredient['Measure_Float'])
        if ingredient['Liquid'][:6] == "rose's":
            ingredient['Liquid'] = ingredient['Liquid'][7:]
        elif ingredient['Liquid'][:8] =="peychaud": 
            ingredient['Liquid'] = "peychauds bitters"
    recipe['Total_Volume'] = sum(vols)
        

In [17]:
for recipe in all_recipes_copy:
    recipe['Cocktail_Name'] = recipe.pop('name')
    recipe['Category'] = recipe.pop('category')
    recipe['Instructions'] = recipe.pop('instructions')
    recipe['Glass_Size'] = recipe.pop('glass_size')
    recipe['Glass'] = recipe.pop('glass')
    recipe.pop('recipe')
        

In [18]:
all_recipes_copy[2]

{'Ingredients': [{'Liquid': 'apple schnapps',
   'Measure': '3 oz',
   'Measure_Float': 3.0},
  {'Liquid': 'cinnamon schnapps', 'Measure': '1 oz', 'Measure_Float': 1.0},
  {'Liquid': 'cinnamon schnapps', 'Measure': '1 oz', 'Measure_Float': 1.0}],
 'Garnish_Instructions': 'add',
 'Garnish': 'green apple slice',
 'Total_Volume': 5.0,
 'Cocktail_Name': 'Apple Pie',
 'Category': 'Cordials and Liqueurs',
 'Instructions': 'Pour into ice-filled old-fashioned glass. Garnish with apple and top with cinnamon.',
 'Glass_Size': 6,
 'Glass': 'Old-Fashioned Glass'}

In [20]:
#AI Recipes load

#create glass list
glasses = []
for i in range(len(all_recipes_copy)):
    glasses.append(all_recipes_copy[i]["Glass"])
glasses = list(set(glasses))

df = pd.DataFrame(all_recipes_copy)

#create glass sizes list
glass_sizes = []
for glass in glasses:
    sizes = df.loc[df["Glass"] == glass, "Glass_Size"]
    glass_sizes.append(list(sizes)[0])
    
#create glass size dictionary
glass_size_dict = {}
for i in range(len(glasses)):
    glass_size_dict[glasses[i]] = glass_sizes[i]
glass_size_dict

text_files = ["MACKs_AI_Classics_2.txt", "MACKs_AI_Classics_5.txt", "MACKs_AI_Classics_10.txt"]

def parse_ai_text(text_file):
    #to track how many records are dropped due to what criteria
    all_lines = 0
    good_cocktails = 0
    too_many_ingredients = 0
    wrong_glass = 0
    bad_instructions = 0
    bad_volume = 0
    
    #open text file to read
    with open(text_file, "r") as file:
        for line in file:
    #         print(line)
            #split line
            split_line = line.split(" - ")
            all_lines += 1

            #get name
            name = split_line[0]
            print(f"COCKTAIL NAME: {name}")

            #get glass
            glass = split_line[1]
            print(f"GLASS NAME: {glass}")

            #get instructions
            instructions = split_line[-1:][0][:-1]
            instructions = re.sub(r'["]', '\'', instructions)
            print(f"INSTRUCTIONS: {instructions}")


            garnish_list = []
            measure_list = []
            measure_float_list = []
            liquid_list = []
            total_volume = 0
            try:
                glass_volume = glass_size_dict[glass]
            except KeyError:
                glass_volume = 0
            
            recipe = {}

            #get number of ingredients (split length minus name, glass, instructions)
            ingredients_length = len(split_line) - 3
            #loop over ingredients
            for i in range(3, ingredients_length + 2):
                this_ing = {}
                ingredient = split_line[i]

                #if ingredient begins with 'add', its a garnish
                if ingredient[:3] == "add":
                    recipe['Garnish'] = ingredient.split("add ")[1]
                    recipe['Garnish_Instructions'] = ingredient.split("add ")[0]
                    
                    

                #if ingredient starts with 'fill',
                elif split_line[i][:4] == "fill":
                    measure_list.append("fill")
                    print("MEASURE: fill")

                    #get liquid ingredient
                    liquid = split_line[i].split("fill ")[1]
                    liquid_list.append(liquid.strip())
                    print(f"LIQUID NAME: {liquid}")

                    #calculate measure_float
                    remaining_volume = float(glass_volume) - float(total_volume)
                    measure_float = float(remaining_volume / 2)
                    measure_float_list.append(measure_float)
                    print(f"MEASURE FLOAT: {measure_float}")

                   


                else:
                    try:
                        #if ingredient begins with number, its a meausure and a liquid
                        int(ingredient[:1])
                    except ValueError:
                        pass
                    #split ingredient on spaces
                    split_ingredient = ingredient.split(" ")                

                    #loop over splits
                    for j in range(len(split_ingredient)):
                        #if split is unit of measure, check for compound fraction
                        if split_ingredient[j] in ["oz", "dash", "splash", "tsp"]:
                            units = split_ingredient[j].strip()                        

                            #if j-2 > -1, there is a compound fraction
                            if j-2 > -1:
                                #get 2 parts of compound fraction
                                fraction = split_ingredient[j-1].strip()
                                whole_number = split_ingredient[j-2].strip()                           

                                #recombine compound fraction
                                measure = whole_number+ " " +fraction
                                measure_list.append(measure + " " + units)
                                print(f"MEASURE: {measure + ' ' + units}")

                                #get liquid ingredient
                                liquid = ingredient.split(measure + " " + units)[1]
                                liquid_list.append(liquid.strip())                            
                                print(f"LIQUID: {liquid}")

                                #convert measure to float
                                try:
                                    measure_float = float(sum(Fraction(s) for s in measure.split()))
                                except ValueError:
                                    pass

                                #if unit is smaller than oz, convert measure to oz
                                if ingredient[:3] == "tsp":
                                    measure_float = measure_float / 6
                                elif ingredient[:4] == "dash":
                                    measure_float = measure_float / 32
                                elif ingredient[:6] == "splash":
                                    measure_float = measure_float / 5
                                measure_float_list.append(measure_float)
                                print(f"MEASURE FLOAT: {measure_float}")

                                #add measure to total volume
                                total_volume += measure_float

                            else: #just a single number or fraction
                                #get measure
                                measure = split_ingredient[j-1].strip()
                                measure_list.append(measure + " " + units)
                                print(f"MEASURE: {measure + ' ' + units}")


                                #get liquid ingredient
                                liquid = ingredient.split(measure + " " + units)[1]
                                liquid_list.append(liquid.strip())
                                print(f"LIQUID: {liquid}")

                                #convert to float
                                measure_float = float(sum(Fraction(s) for s in measure.split()))
                                #if unit is smaller than oz, convert measure to oz
                                if ingredient[:3] == "tsp":
                                    measure_float = measure_float / 6
                                elif ingredient[:4] == "dash":
                                    measure_float = measure_float / 32
                                elif ingredient[:6] == "splash":
                                    measure_float = measure_float / 5
                                measure_float_list.append(measure_float)
                                print(f"MEASURE FLOAT: {measure_float}")

                                #add measure to total volume    
                                total_volume += measure_float
            print("731 # 731 # 731")

            #if more than 6 ingredients, skip
            if len(split_line) > 10:
                print("INGREDIENTS ERRORRRRRRRRRRR")
                too_many_ingredients += 1
                pass

            elif glass not in glasses:
                print("GLASS ERRORRRRRRRRRR")
                wrong_glass += 1
                pass

            elif instructions[:3] == "add":
                print("INSTRUCTIONS ERRORRRRRRRRR")
                bad_instructions += 1
                pass 

            elif total_volume > glass_volume:
                print("VOLUME ERRORRRRRRRRRRRR")
                bad_volume += 1

            else:
                print("+++++++++++++++++++++++ GOOOD ++++++++++++++++++++")
                good_cocktails += 1
                
                #insert cocktail name and instructions into cocktail table
                recipe['Cocktail_Name'] = name
                print(name)
                recipe['Instructions'] = instructions
                recipe['Glass'] = glass
                recipe['Category'] = 'AI Instant Classic'
                
                
                recipe['Ingredients'] = []
                for i in range(len(liquid_list)):
                    this_ing = {}
                    this_ing['Liquid'] = liquid_list[i]
                    this_ing['Measure'] = measure_list[i]
                    this_ing['Measure_Float'] = measure_float_list[i]
                    recipe['Ingredients'].append(this_ing)
                print(recipe)
                all_recipes_copy.append(recipe)
    
            print("----------------------")
    

    # print(all_lines, too_many_ingredients, wrong_glass, bad_instructions, bad_volume, good_cocktails)
    print("####################################")


for file in text_files:
    parse_ai_text(file)

COCKTAIL NAME: Black Hawk
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass. Add
MEASURE: 1 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 1.0
MEASURE: 1/2 oz
LIQUID:  simple syrup
MEASURE FLOAT: 0.5
MEASURE: 1 oz
LIQUID:  ginger ale or soda water
MEASURE FLOAT: 1.0
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Black Hawk
{'Garnish': 'pineapple wedge skewered with maraschino cherry', 'Garnish_Instructions': '', 'Cocktail_Name': 'Black Hawk', 'Instructions': 'Shake with ice and strain into chilled cocktail glass. Add', 'Glass': 'Cocktail Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'fresh lemon juice', 'Measure': '1 oz', 'Measure_Float': 1.0}, {'Liquid': 'simple syrup', 'Measure': '1/2 oz', 'Measure_Float': 0.5}, {'Liquid': 'ginger ale or soda water', 'Measure': '1 oz', 'Measure_Float': 1.0}]}
----------------------
COCKTAIL NAME: Biffy Cocktail
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake first fo

INSTRUCTIONS: Shake with ice and strain into cocktail glass.
MEASURE: 3 oz
LIQUID:  coconut milk
MEASURE FLOAT: 3.0
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Saronno Colada (Nutty Colada)
{'Cocktail_Name': 'Saronno Colada (Nutty Colada)', 'Instructions': 'Shake with ice and strain into cocktail glass.', 'Glass': 'Collins Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'coconut milk', 'Measure': '3 oz', 'Measure_Float': 3.0}]}
----------------------
COCKTAIL NAME: Saronno Colada (Nutty Colada)
GLASS NAME: Collins Glass
INSTRUCTIONS: Shake with ice and strain into cocktail glass.
MEASURE: 3 oz
LIQUID:  coconut milk
MEASURE FLOAT: 3.0
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Saronno Colada (Nutty Colada)
{'Cocktail_Name': 'Saronno Colada (Nutty Colada)', 'Instructions': 'Shake with ice and strain into cocktail glass.', 'Glass': 'Collins Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'coconut milk

731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Bourbon Cooler
{'Garnish': 'egg white', 'Garnish_Instructions': '', 'Cocktail_Name': 'Bourbon Cooler', 'Instructions': 'Shake with ice and strain into cocktail glass.', 'Glass': 'Old-Fashioned Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'white creme de cacao', 'Measure': '1 oz', 'Measure_Float': 1.0}]}
----------------------
COCKTAIL NAME: Bourbon Cobbler
GLASS NAME: Old-Fashioned Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass.
MEASURE: 1 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 1.0
MEASURE: 1 oz
LIQUID:  grenadine
MEASURE FLOAT: 1.0
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Bourbon Cobbler
{'Cocktail_Name': 'Bourbon Cobbler', 'Instructions': 'Shake with ice and strain into chilled cocktail glass.', 'Glass': 'Old-Fashioned Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'fresh lemon juice', 'Measure': '1 oz', 'Measure_F

INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass.
MEASURE: 1 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 1.0
MEASURE: 1 oz
LIQUID:  simple syrup
MEASURE FLOAT: 1.0
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
White Lion Cocktail
{'Cocktail_Name': 'White Lion Cocktail', 'Instructions': 'Shake with ice and strain into chilled cocktail glass.', 'Glass': 'Cocktail Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'fresh lemon juice', 'Measure': '1 oz', 'Measure_Float': 1.0}, {'Liquid': 'simple syrup', 'Measure': '1 oz', 'Measure_Float': 1.0}]}
----------------------
COCKTAIL NAME: Hurricane Leah
GLASS NAME: Hurricane Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass. Add grapefruit twist.
MEASURE: 1 1/2 oz
LIQUID:  orange flavored gin
MEASURE FLOAT: 1.5
MEASURE: 1/4 oz
LIQUID:  apricot flavored brandy
MEASURE FLOAT: 0.25
MEASURE: 1/4 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 0.25
731 # 731 # 731
+++++++

MEASURE FLOAT: 0.75
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Sunshine Cocktail
{'Cocktail_Name': 'Sunshine Cocktail', 'Instructions': 'Shake with ice and strain into chilled cocktail glass. Garnish with strawberry slice.', 'Glass': 'Cocktail Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'white creme de cacao', 'Measure': '3/4 oz', 'Measure_Float': 0.75}, {'Liquid': 'lillet rouge', 'Measure': '3/4 oz', 'Measure_Float': 0.75}]}
----------------------
COCKTAIL NAME: Polynesian Cocktail
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass.
MEASURE: 1 oz
LIQUID:  cherry  flavored brandy
MEASURE FLOAT: 1.0
MEASURE: 1/2 oz
LIQUID:  orange flavored gin
MEASURE FLOAT: 0.5
MEASURE: 1/2 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 0.5
MEASURE: 1/2 oz
LIQUID:  simple syrup
MEASURE FLOAT: 0.5
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Polynesian Cocktail
{'Garnish': 'orange and lemon w

...

MEASURE: 1 oz
LIQUID:  hot red pepper sauce
MEASURE FLOAT: 1.0
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Antoine Special
{'Garnish': 'tomato catsup', 'Garnish_Instructions': '', 'Cocktail_Name': 'Antoine Special', 'Instructions': 'Shake with ice and pour into chilled cocktail glass.', 'Glass': 'Old-Fashioned Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'balsamic vinegar', 'Measure': '1 oz', 'Measure_Float': 1.0}, {'Liquid': 'hot red pepper sauce', 'Measure': '1 oz', 'Measure_Float': 1.0}]}
----------------------
COCKTAIL NAME: Saronno Colada (Nutty Colada)
GLASS NAME: Collins Glass
INSTRUCTIONS: Shake with ice and strain into cocktail glass.
MEASURE: 1 oz
LIQUID:  dubonnet
MEASURE FLOAT: 1.0
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Saronno Colada (Nutty Colada)
{'Cocktail_Name': 'Saronno Colada (Nutty Colada)', 'Instructions': 'Shake with ice and strain into cocktail glass.', 'Glass': 'Collins Glass', 'Category'

MEASURE: 3/4 oz
LIQUID:  maraschino liqueur
MEASURE FLOAT: 0.75
MEASURE: 3/4 oz
LIQUID:  cherry  flavored brandy
MEASURE FLOAT: 0.75
MEASURE: 1/2 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 0.5
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Robin's Nest
{'Cocktail_Name': "Robin's Nest", 'Instructions': 'Shake with ice and strain into cocktail glass.', 'Glass': 'Cocktail Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'maraschino liqueur', 'Measure': '3/4 oz', 'Measure_Float': 0.75}, {'Liquid': 'cherry  flavored brandy', 'Measure': '3/4 oz', 'Measure_Float': 0.75}, {'Liquid': 'fresh lemon juice', 'Measure': '1/2 oz', 'Measure_Float': 0.5}]}
----------------------
COCKTAIL NAME: Rum Cobbler
GLASS NAME: Red-Wine Glass
INSTRUCTIONS: Stir with ice and strain into cocktail glass.
MEASURE: 3/4 oz
LIQUID:  dry vermouth
MEASURE FLOAT: 0.75
MEASURE: 3/4 oz
LIQUID:  orange flavored gin
MEASURE FLOAT: 0.75
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++

INSTRUCTIONS: Serve in an old-fashioned glass over crushed ice with a twist of lemon or a wedge of lime, if desired.
MEASURE: 1/4 oz
LIQUID:  amaretto di saronno
MEASURE FLOAT: 0.25
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Oldest Temptation
{'Cocktail_Name': 'Oldest Temptation', 'Instructions': 'Serve in an old-fashioned glass over crushed ice with a twist of lemon or a wedge of lime, if desired.', 'Glass': 'Cocktail Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'amaretto di saronno', 'Measure': '1/4 oz', 'Measure_Float': 0.25}]}
----------------------
COCKTAIL NAME: The Mangarite
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Pour carefully, in order given into collins glass and stir briefly. Fill with ice. Garnish with rosemary.
MEASURE: 1/2 oz
LIQUID:  white creme de cacao
MEASURE FLOAT: 0.5
MEASURE: 1/2 oz
LIQUID:  benedictine
MEASURE FLOAT: 0.5
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
The Mangarite
{'Cocktail_Nam

{'Garnish': 'strawberry slice', 'Garnish_Instructions': '', 'Cocktail_Name': '400imimor', 'Instructions': 'Muddle sage in mixing glass. Add remaining ingredients. Shake with ice and strain through barspoon until glass is ', 'Glass': 'Old-Fashioned Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'fresh lime juice', 'Measure': '3/4 oz', 'Measure_Float': 0.75}, {'Liquid': 'simple syrup', 'Measure': '1/2 oz', 'Measure_Float': 0.5}, {'Liquid': 'ginger ale or soda water', 'Measure': 'fill', 'Measure_Float': 2.375}]}
----------------------
COCKTAIL NAME: Mexican Madras
GLASS NAME: Old-Fashioned Glass
INSTRUCTIONS: A Martini substituting an olive stuffed with an almond for the regular olive. See Martini on page 130.
MEASURE: 1 1/2 oz
LIQUID:  wild cherry flavored brandy
MEASURE FLOAT: 1.5
MEASURE: 3/4 oz
LIQUID:  orange flavored gin
MEASURE FLOAT: 0.75
MEASURE: 3/4 oz
LIQUID:  dry vermouth
MEASURE FLOAT: 0.75
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++

+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Bloodhound Cocktail
{'Garnish': 'orange twist', 'Garnish_Instructions': '', 'Cocktail_Name': 'Bloodhound Cocktail', 'Instructions': 'Shake with ice and strain into chilled cocktail glass.', 'Glass': 'Cocktail Glass', 'Category': 'AI Instant Classic', 'Ingredients': [{'Liquid': 'simple syrup', 'Measure': '1/2 oz', 'Measure_Float': 0.5}]}
----------------------
COCKTAIL NAME: Catalina Margarita
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into highball glass over two ice cubes. Fill with carbonated water and stir.
MEASURE: 1/2 oz
LIQUID:  triple sec
MEASURE FLOAT: 0.5
MEASURE: 1/4 oz
LIQUID:  simple syrup
MEASURE FLOAT: 0.25
731 # 731 # 731
+++++++++++++++++++++++ GOOOD ++++++++++++++++++++
Catalina Margarita
{'Cocktail_Name': 'Catalina Margarita', 'Instructions': 'Shake with ice and strain into highball glass over two ice cubes. Fill with carbonated water and stir.', 'Glass': 'Cocktail Glass', 'Category': 'AI Insta

In [ ]:
good_cocktails

In [35]:
count = 0
for recipe in all_recipes_copy:
    while count < 1:
        if recipe['Category'] == 'AI Instant Classic':
            print(recipe)
            count += 1

KeyboardInterrupt: 

In [32]:
color_dict_list[0]


{'liquids': '100-proof vodka', 'color': '#fbfff8'}

In [30]:
color_dict_list = liquid_colors.to_dict(orient='records')
liquid_list = list(liquid_colors['liquids'])

In [33]:
for recipe in all_recipes_copy:
    for ingredient in recipe['Ingredients']:
        for color_dict in color_dict_list:
            if ingredient['Liquid'] not in liquid_list:
                #generate random hex color
                ingredient['Color'] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
                #insert new liquid and color into liquids table
            else:
                if ingredient['Liquid'] == color_dict['liquids']:
                    ingredient['Color'] = color_dict['color']


In [34]:
all_recipes_copy[0]

{'Ingredients': [{'Liquid': 'batavia arrack or light rum',
   'Measure': '2 oz',
   'Measure_Float': 2.0,
   'Color': '#eedfcc'},
  {'Liquid': 'passion fruit syrup',
   'Measure': '1 oz',
   'Measure_Float': 1.0,
   'Color': '#deff62'},
  {'Liquid': 'fresh lemon juice',
   'Measure': '1 oz',
   'Measure_Float': 1.0,
   'Color': '#deff62'},
  {'Liquid': 'lemon or lime juice',
   'Measure': '1 oz',
   'Measure_Float': 1.0,
   'Color': '#deff62'}],
 'Total_Volume': 5.0,
 'Cocktail_Name': 'Gauguin',
 'Category': 'Cocktail Classics',
 'Instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Top with a cherry.',
 'Glass_Size': 6,
 'Glass': 'Old-Fashioned Glass'}